In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
cwd = os.getcwd()
cancel = pd.read_csv(cwd + '/cancel.csv')

"A" = "Carrier" <br> 
> The cause of the cancellation or delay was due to circumstances within the airline's control (e.g. maintenance or crew problems, etc.).<br>

"B" = "Weather" <br> 
> Significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane.<br>

"C" = "National Air System" <br>
> Cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control.<br>

"D" = "Security" <br>
> Cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.<br>

In [3]:
# cancel['Freq'] = cancel.groupby(['ORIGIN'])['OP_UNIQUE_CARRIER'].transform('count')

In [3]:
D = cancel[cancel['CANCELLATION_CODE'] == 'D']

# cancel = cancel[cancel['Freq'] > 10] 
#removed any airport used less than 10 times within the year
# excpet for D bec so few security problems
# went from 110320 rows to 109983 rows
A = cancel[cancel['CANCELLATION_CODE'] == 'A']
B = cancel[cancel['CANCELLATION_CODE'] == 'B']
C = cancel[cancel['CANCELLATION_CODE'] == 'C']

## CODE: A (Carrier)

In [94]:
# A has 28085 rows
# 273 unique airports

In [5]:
#certain planes that might be more prone to canceltaion
A['TAIL_NUM'].value_counts()[:15]

N719EV    66
N738EV    60
N226JS    52
N248PS    51
N603QX    49
N244PS    48
N250PS    48
N258PS    48
N251PS    47
N617QX    47
N600QX    45
N611QX    44
N230PS    42
N720EV    42
N223JS    41
Name: TAIL_NUM, dtype: int64

In [11]:
A['OP_UNIQUE_CARRIER'].value_counts()

WN    5372
OH    4425
AA    4176
YV    2216
YX    1663
MQ    1519
F9    1476
B6    1377
EV    1195
AS     985
UA     891
OO     772
DL     565
G4     396
NK     331
9E     314
HA     256
VX     224
Name: OP_UNIQUE_CARRIER, dtype: int64

In [9]:
A['MONTH'].value_counts()

6     5959
7     3391
8     3094
3     2348
1     2318
5     2201
9     1881
4     1867
2     1726
10    1638
12     990
11     672
Name: MONTH, dtype: int64

In [15]:
A['ORIGIN'].value_counts()[:5]

CLT    1939
DFW    1406
LAX    1034
LGA     975
ORD     815
Name: ORIGIN, dtype: int64

In [16]:
A['DEST'].value_counts()[:5]

CLT    2388
DFW    1740
LAX    1068
ORD    1025
LGA    1008
Name: DEST, dtype: int64

In [4]:
# get frequency of canceled airline and airport combo
J = A.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN']).size()
aline = A['OP_UNIQUE_CARRIER'].unique()
aport = A['ORIGIN'].unique()
amonth = A['MONTH'].unique()

In [6]:
a_combo = []
for l in aline:
    for a in aport:
        if a in J[l]:
            if J[l][a] > 100:
                a_combo.append([J[l][a], l, a])

In [15]:
worst_a_line_port = sorted(a_combo, reverse=True)

In [16]:
worst_a_line_port[:5] 

[[1363, 'OH', 'CLT'],
 [572, 'WN', 'LAX'],
 [498, 'AA', 'DFW'],
 [495, 'YV', 'DFW'],
 [434, 'WN', 'OAK']]

In [17]:
# get frequency of canceled airline, airport and month combo
M = A.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN', 'MONTH']).size()

In [18]:
a_trio = []
for l in aline:
    for a in aport:
        if a in M[l]:
            for m in range(1,12):
                if m in M[l][a]:
                    if M[l][a][m] > 50:
                        a_trio.append([M[l][a][m], l, a, m])

In [19]:
worst_a_line_port_month = sorted(a_trio, reverse=True)

In [21]:
worst_a_line_port_month[:5] 

[[1161, 'OH', 'CLT', 6],
 [180, 'OH', 'DCA', 6],
 [147, 'AA', 'DFW', 6],
 [131, 'OH', 'PHL', 6],
 [113, 'YX', 'LGA', 3]]

In [ ]:
# giant glitch with computer sytem and PSA airlines ('OH') during June
# mainly in Charlotte, this skewed a lot of data for cancellation

## CODE: B (Weather)

In [ ]:
# 57756
# 338 airports from

In [41]:
38878/57756

0.673142184361798

In [39]:
B['MONTH'].value_counts()
# About 67% of weather cancellations happened within the first three months of the year
# showing winter is the strongest factor for these cancellations

3     13566
1     12656
2      5753
9      4609
7      3479
4      3233
5      3217
8      3159
12     2947
6      2486
10     2147
11      504
Name: MONTH, dtype: int64

In [37]:
B['OP_UNIQUE_CARRIER'].value_counts()

WN    9774
AA    8535
OO    7760
OH    5362
YX    4813
DL    3864
B6    3601
UA    3265
MQ    2204
YV    2105
9E    2009
EV    1389
NK    1289
F9     752
AS     576
G4     243
VX     201
HA      14
Name: OP_UNIQUE_CARRIER, dtype: int64

In [9]:
B['ORIGIN'].value_counts()[:10]

ORD    3295
LGA    3029
CLT    2869
DFW    2592
BOS    2393
ATL    2327
EWR    2181
DCA    2046
PHL    1857
MDW    1558
Name: ORIGIN, dtype: int64

In [12]:
B['DEST'].value_counts()[:10]

ORD    3442
CLT    2963
LGA    2929
DFW    2661
ATL    2400
BOS    2345
EWR    2211
DCA    1961
PHL    1857
JFK    1535
Name: DEST, dtype: int64

In [11]:
B['ORIGIN_CITY_NAME'].value_counts()[:10]

Chicago, IL              4853
New York, NY             4548
Charlotte, NC            2869
Dallas/Fort Worth, TX    2592
Washington, DC           2535
Boston, MA               2393
Atlanta, GA              2327
Newark, NJ               2181
Philadelphia, PA         1857
Minneapolis, MN          1312
Name: ORIGIN_CITY_NAME, dtype: int64

In [31]:
B['DEST_CITY_NAME'].value_counts()[:10]

Chicago, IL              4971
New York, NY             4464
Charlotte, NC            2963
Dallas/Fort Worth, TX    2661
Washington, DC           2457
Atlanta, GA              2400
Boston, MA               2345
Newark, NJ               2211
Philadelphia, PA         1857
Houston, TX              1360
Name: DEST_CITY_NAME, dtype: int64

In [43]:
not_cold = B[(B['ORIGIN_CITY_NAME'] != 'Chicago, IL') &(B['ORIGIN_CITY_NAME'] != 'New York, NY')\
             & (B['DEST_CITY_NAME'] !='Chicago, IL')&(B['DEST_CITY_NAME'] !='New York, NY')]

In [75]:
city = B.reindex(columns= (['ORIGIN_CITY_NAME', 'DEST_CITY_NAME']))

In [85]:
city.melt()['value'].value_counts()[:10]

Chicago, IL              9824
New York, NY             9012
Charlotte, NC            5832
Dallas/Fort Worth, TX    5253
Washington, DC           4992
Boston, MA               4738
Atlanta, GA              4727
Newark, NJ               4392
Philadelphia, PA         3714
Houston, TX              2663
Name: value, dtype: int64

In [ ]:
# 31% of cancellations involved Chicago or New York

In [86]:
A.groupby(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'MONTH']).size()

ORIGIN_CITY_NAME  DEST_CITY_NAME         MONTH
Abilene, TX       Dallas/Fort Worth, TX  1         1
                                         2         1
                                         5         1
                                         6         1
                                         8         2
Aguadilla, PR     Fort Lauderdale, FL    3         1
                  New York, NY           1         1
                                         7         1
                  Orlando, FL            7         1
Akron, OH         Charlotte, NC          1         1
                                         2         1
                                         3         1
                                         4         2
                                         5         1
                                         6        16
                                         7         4
                                         8         1
                                         9         1

## CODE: C

## CODE: D

In [28]:
len(D)

73